In [1]:
import os
import time
import math
import json
import redis
import random
import asyncio
import traceback
from datetime import datetime
from aredis import StrictRedis

In [2]:
from pathlib import Path
os.chdir(Path(os.getcwd()).parent)

In [3]:
from libs.utils import Utils
from libs.dailydata import DailyData
from libs.cython.compute import compute_stats

In [4]:
rd = redis.Redis(host='127.0.0.1', port=6379, db=8)
ar = StrictRedis(host='127.0.0.1', port=6379, db=8)

In [5]:
Utils.update_symbols()
symbols = Utils.get_running_symbols()
assist_count = math.ceil(len(symbols)/800)+1
rd.set(f'hq_assist_count', assist_count)

True

In [6]:
date = time.strftime('%Y%m%d')
date = '20210810'
date

'20210810'

In [7]:
dd = DailyData.load(dt=date)

In [17]:
dd.close_sharedmemory()
dd.unlink_sharedmemory()

In [8]:
async def start_fake_data_generator(assist_count=7, snapshot_interval=5):
    assist_indices = list([ _ for _ in range(assist_count)])
    
    for _, check_point in enumerate(dd.check_points):
        random.shuffle(assist_indices)
        try:
            time_lapse = dd.get_time_lapse(_)
            ma5pm_anchor_idx = dd.get_ma5pm_anchor_idx(_)
            fs5p = dd.snapshots[ma5pm_anchor_idx]
            compute_stats(dd.snapshots[_], dd.basics, dd.statistic[_], fs5p, time_lapse)

            for assist_idx in assist_indices:
                await ar.publish(
                    f'hq_assist_{assist_idx}_snapshotting', 
                    json.dumps({"status":'successful',"date":date,"idx":_,'check_point':int(check_point)})
                )
                
            print(str(_).ljust(5, ' '), datetime.fromtimestamp(dd.check_points[_]).strftime('%Y-%m-%d %H:%M:%S'), 'sussess', end='\r')

        except Exception as e:
            error = {
                "status": 'failed',
                "idx": _,
                'check_point':int(check_point),
                "exception": str(e),
                'traceback': traceback.format_exc()
            }

            for assist_idx in assist_indices:
                await ar.publish(
                    f'hq_assist_{assist_idx}_snapshotting', 
                    json.dumps(error)
                )
            print(str(_).ljust(5, ' '), datetime.fromtimestamp(dd.check_points[_]).strftime('%Y-%m-%d %H:%M:%S'), 'failed', str(e), end='\r')
        finally:
            if _ > 190:
                await asyncio.sleep(snapshot_interval)


In [9]:
task = asyncio.create_task(start_fake_data_generator())

In [17]:
task.cancel()

True

In [11]:
import numpy as np

In [14]:
_ = 3006

In [16]:
dd.get_snapshot(_)

,datetime,timestamp,name,open,close,now,high,low,turnover,volume,bid1,bid1_volume,zhangfu,junjia,liangbi,zhangsu,tingban,ma5
symbol,,,,,,,,,,,,,,,,,,
000001,08:50:06,3006,平安银行,17.91,17.86,19.06,19.25,17.84,"175,279,800.00","3,313,803,230.96",19.06,"57,388.00",6.72,18.91,2.20,-0.16,0.00,18.06
000002,08:50:06,3006,万 科Ａ,21.00,21.10,22.28,22.76,20.98,"138,908,123.00","3,081,214,270.26",22.27,"44,700.00",5.59,22.18,1.83,-0.13,0.00,21.16
000004,08:50:06,3006,国华网安,21.90,22.31,22.65,22.81,21.69,"5,462,759.00","122,886,445.43",22.64,"2,000.00",1.52,22.50,0.54,0.35,0.00,22.88
000005,08:50:06,3006,ST星源,1.98,1.98,2.05,2.07,1.98,"11,331,887.00","23,091,657.52",2.05,"40,800.00",3.54,2.04,1.04,0.00,0.00,2.03
000006,08:50:06,3006,深振业Ａ,4.54,4.54,4.60,4.68,4.50,"10,007,143.00","46,121,945.80",4.60,"214,857.00",1.32,4.61,1.45,0.00,0.00,4.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688799,08:50:06,3006,华纳药厂,43.39,43.52,44.00,44.10,42.86,"1,372,054.00","59,548,997.00",43.98,"1,200.00",1.10,43.40,0.65,-0.05,0.00,45.39
688800,08:50:06,3006,瑞可达,85.88,87.90,84.00,86.98,78.75,"2,529,922.00","206,744,028.00",83.88,600.00,-4.44,81.72,0.88,1.86,0.00,86.66
688819,08:50:06,3006,天能股份,55.21,55.95,56.25,58.30,54.50,"9,510,473.00","535,466,854.00",56.25,"37,632.00",0.54,56.30,0.63,0.12,0.00,55.31


In [18]:
ma5pm_anchor_idx = dd.get_ma5pm_anchor_idx(_)
ma5pm_anchor_idx

2945

In [19]:
dd.get_snapshot(2945)

,datetime,timestamp,name,open,close,now,high,low,turnover,volume,bid1,bid1_volume,zhangfu,junjia,liangbi,zhangsu,tingban,ma5
symbol,,,,,,,,,,,,,,,,,,
000001,08:49:05,2945,平安银行,17.91,17.86,19.09,19.25,17.84,"173,596,688.00","3,281,710,779.30",19.08,"139,600.00",6.89,18.90,2.23,6.59,0.00,18.07
000002,08:49:05,2945,万 科Ａ,21.00,21.10,22.31,22.76,20.98,"136,288,483.00","3,022,826,808.26",22.30,"230,640.00",5.73,22.18,1.83,6.24,0.00,21.17
000004,08:49:05,2945,国华网安,21.90,22.31,22.57,22.81,21.69,"4,999,159.00","112,394,527.43",22.57,"2,300.00",1.17,22.48,0.51,3.06,0.00,22.86
000005,08:49:05,2945,ST星源,1.98,1.98,2.05,2.07,1.98,"11,051,887.00","22,516,883.52",2.05,"7,600.00",3.54,2.04,1.03,3.54,0.00,2.03
000006,08:49:05,2945,深振业Ａ,4.54,4.54,4.60,4.68,4.50,"9,711,243.00","44,759,294.80",4.60,"88,957.00",1.32,4.61,1.44,1.32,0.00,4.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688799,08:49:05,2945,华纳药厂,43.39,43.52,44.02,44.10,42.86,"1,324,727.00","57,466,796.00",44.02,600.00,1.15,43.38,0.64,1.45,0.00,45.39
688800,08:49:05,2945,瑞可达,85.88,87.90,82.47,86.98,78.75,"2,475,652.00","202,190,389.00",82.48,"1,534.00",-6.18,81.67,0.88,-3.97,0.00,86.35
688819,08:49:05,2945,天能股份,55.21,55.95,56.18,58.30,54.50,"9,328,992.00","525,264,060.00",56.17,"5,576.00",0.41,56.30,0.63,1.76,0.00,55.29


In [23]:
dd.check_points[2945]

1628492100

In [22]:
time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(1628492100))

'2021-08-09 14:55:00'

In [30]:
ms = dd.snapshots[_]
ms

array([[17.91, 17.86, 19.06, ..., 3313803230.96, 19.06, 57388.00],
       [21.00, 21.10, 22.28, ..., 3081214270.26, 22.27, 44700.00],
       [21.90, 22.31, 22.65, ..., 122886445.43, 22.64, 2000.00],
       ...,
       [55.21, 55.95, 56.25, ..., 535466854.00, 56.25, 37632.00],
       [65.79, 67.46, 64.03, ..., 6073021844.00, 64.03, 352935.00],
       [63.00, 62.96, 63.52, ..., 58665194.00, 63.51, 200.00]])

In [28]:
fs5p = dd.snapshots[2945]
fs5p

array([[17.91, 17.86, 19.09, ..., 3281710779.30, 19.08, 139600.00],
       [21.00, 21.10, 22.31, ..., 3022826808.26, 22.30, 230640.00],
       [21.90, 22.31, 22.57, ..., 112394527.43, 22.57, 2300.00],
       ...,
       [55.21, 55.95, 56.18, ..., 525264060.00, 56.17, 5576.00],
       [65.79, 67.46, 63.98, ..., 5988397897.00, 63.97, 6603.00],
       [63.00, 62.96, 63.49, ..., 57157237.00, 63.42, 674.00]])

In [35]:
(ms[4,2]/fs5p[4,0]-1)*

0.013215859030836885

4.54

In [13]:
for key in rd.keys():
    print(key)
    if key.decode("utf-8") .startswith('test'):
        continue
    rd.delete(key)

In [ ]:
int(rd.get(f'hq_{date}_check_points_length'))

In [ ]:
rd.get(f'hq_{date}_check_points_length')

In [120]:
idx = 2661

In [121]:
result = dd.statistic[idx, :, 4] > 0
result

array([False, False,  True, ..., False, False, False])

In [122]:
len(np.argwhere(result))

154

In [123]:
result2 = dd.statistic[idx-1, :, 4] <= 0
result2

array([ True,  True, False, ...,  True,  True,  True])

In [124]:
len(np.argwhere(result2))

4222

In [125]:
np.argwhere(result == result2)

array([[3348]], dtype=int64)

In [126]:
result2[3448]

True

In [119]:
dd.check_points[np.argmax(dd.statistic[:2703, 3348, 4] > 0)]

1626071480

In [ ]:
time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(float(ts)))

In [ ]:
dd.statistic[1000,:,4]

In [ ]:
import numpy as np

In [ ]:
[
    [
        idx[0], 
        dd.check_points[np.argmax(dd.statistic[:,idx[0],4]>0)]
    ] for idx in np.argwhere(dd.statistic[2000,:,4]>0)
]


In [ ]:
start = time.time()
json.dumps(list([
    (
        int(idx[0]), 
        int(dd.check_points[np.argmax(dd.statistic[:,idx[0],4]>0)])
    ) for idx in np.argwhere(dd.statistic[2103,:,4]>0)
]))
time.time()-start

In [ ]:
delta = 1926
time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(1626073210+delta))

In [ ]:
dd.check_points[2103]

In [ ]:
1626052500+delta

In [ ]:
dd.check_points[-1]

In [ ]:
import numpy as np

In [ ]:
np.argwhere(dd.statistic[1358, :, 4] > 0).tolist()

In [ ]:
dd.check_points[1358]